# **GMM Model**

In [ ]:
import numpy as np
import pandas as pd
import os
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from sklearn.mixture import GaussianMixture

# List of leaders' names
leaders = ['Nelson_Mandela', 'Benjamin_Netanyau', 'Magaret_Tarcher', 'Jens_Stoltenberg', 'Julia_Gillard']

# Path to the dataset
data_dir = '/kaggle/input/speaker-recognition-dataset/16000_pcm_speeches'

# Initialize lists to store file paths and labels
file_paths = []
labels = []

# Loop over the directories and files
for dirname, _, filenames in os.walk(data_dir):
    for filename in filenames:
        # Get the full file path
        file_path = os.path.join(dirname, filename)
        
        # Check if the file belongs to one of the leaders' classes
        speaker = os.path.basename(dirname)
        if speaker in leaders:
            file_paths.append(file_path)
            labels.append(speaker)

# Convert labels to numerical values
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(file_paths, labels, test_size=0.2)

# MFCC feature extraction
def extract_features(file_path):
    audio, sr = librosa.load(file_path, sr=None)
    energy = np.sum(audio**2)
    return energy

# Extract features for train and test sets
X_train_features = np.array([extract_features(file_path) for file_path in X_train])
X_test_features = np.array([extract_features(file_path) for file_path in X_test])

# Reshape features for GMM
X_train_flat = X_train_features.reshape(X_train_features.shape[0], -1)
X_test_flat = X_test_features.reshape(X_test_features.shape[0], -1)

# Number of Gaussian components
num_components = len(leaders)  # Assuming one component per speaker

# Initialize GMM
gmm = GaussianMixture(n_components=num_components, covariance_type='diag', random_state=42)

# Fit GMM to the flattened MFCC features
gmm.fit(X_train_flat)

# Predict speaker labels for test set
predicted_labels = gmm.predict(X_test_flat)

# Evaluate accuracy
accuracy = np.mean(predicted_labels == y_test)
print("Accuracy:", accuracy)

In [ ]:
import numpy as np
import pandas as pd
import os
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from sklearn.mixture import GaussianMixture

# List of leaders' names
leaders = ['Nelson_Mandela', 'Benjamin_Netanyau', 'Magaret_Tarcher', 'Jens_Stoltenberg', 'Julia_Gillard']

# Path to the dataset
data_dir = '/kaggle/input/speaker-recognition-dataset/16000_pcm_speeches'

# Initialize lists to store file paths and labels
file_paths = []
labels = []

# Loop over the directories and files
for dirname, _, filenames in os.walk(data_dir):
    for filename in filenames:
        # Get the full file path
        file_path = os.path.join(dirname, filename)
        
        # Check if the file belongs to one of the leaders' classes
        speaker = os.path.basename(dirname)
        if speaker in leaders:
            file_paths.append(file_path)
            labels.append(speaker)

# Convert labels to numerical values
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(file_paths, labels, test_size=0.2)

# MFCC feature extraction
def extract_features(file_path):
    audio, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    mfccs = mfccs.T
    return mfccs
    return mfcc

# Extract features for train and test sets
X_train_features = np.array([extract_features(file_path) for file_path in X_train])
X_test_features = np.array([extract_features(file_path) for file_path in X_test])

# Reshape features for GMM
X_train_flat = X_train_features.reshape(X_train_features.shape[0], -1)
X_test_flat = X_test_features.reshape(X_test_features.shape[0], -1)

# Number of Gaussian components
num_components = len(leaders)  # Assuming one component per speaker

# Initialize GMM
gmm = GaussianMixture(n_components=num_components, covariance_type='diag', random_state=42)

# Fit GMM to the flattened MFCC features
gmm.fit(X_train_flat)

# Predict speaker labels for test set
predicted_labels = gmm.predict(X_test_flat)

# Evaluate accuracy
accuracy = np.mean(predicted_labels == y_test)
print("Accuracy:", accuracy)

In [1]:
import numpy as np
import pandas as pd
import os
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.mixture import GaussianMixture

# List of leaders' names
leaders = ['Nelson_Mandela', 'Benjamin_Netanyau', 'Magaret_Tarcher', 'Jens_Stoltenberg', 'Julia_Gillard']

# Path to the dataset
data_dir = '/kaggle/input/speaker-recognition-dataset/16000_pcm_speeches'

# Initialize lists to store file paths and labels
file_paths = []
labels = []

# Loop over the directories and files
for dirname, _, filenames in os.walk(data_dir):
    for filename in filenames:
        # Get the full file path
        file_path = os.path.join(dirname, filename)
        
        # Check if the file belongs to one of the leaders' classes
        speaker = os.path.basename(dirname)
        if speaker in leaders:
            file_paths.append(file_path)
            labels.append(speaker)

# Convert labels to numerical values
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(file_paths, labels, test_size=0.2, random_state=42)

# Define a function to extract MFCC features
def extract_features(file_path):
    audio, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    return mfccs.T

# Initialize a dictionary to store GMMs for each speaker
gmm_models = {}

# Train a separate GMM for each speaker
for leader in leaders:
    # Extract features for files belonging to the current speaker
    X_train_speaker = [extract_features(file_path) for file_path, label in zip(X_train, y_train) if label == leaders.index(leader)]
    X_train_speaker_flat = np.vstack(X_train_speaker)
    
    # Initialize GMM for the current speaker
    gmm = GaussianMixture(n_components=5, covariance_type='diag', random_state=42)
    
    # Fit GMM to the flattened MFCC features
    gmm.fit(X_train_speaker_flat)
    
    # Store the trained GMM in the dictionary
    gmm_models[leader] = gmm

# Predict speaker labels for test set
predicted_labels = []
for file_path in X_test:
    # Extract features for the current test file
    test_features = extract_features(file_path)
    
    # Initialize variables to keep track of the predicted speaker and the highest log likelihood
    predicted_speaker = None
    max_log_likelihood = -np.inf
    
    # Predict speaker label using the GMM for each speaker and select the one with the highest log likelihood
    for leader, gmm in gmm_models.items():
        log_likelihood = gmm.score(test_features)
        if log_likelihood > max_log_likelihood:
            max_log_likelihood = log_likelihood
            predicted_speaker = leader
    
    # Append the predicted label to the list
    predicted_labels.append(leaders.index(predicted_speaker))

# Evaluate accuracy
accuracy = np.mean(predicted_labels == y_test)
print("Accuracy:", accuracy)

Accuracy: 0.9820119920053297


# **RNN Model**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import shutil
import matplotlib.pyplot as plt
import numpy as np

Combining files of each speaker for visualization

In [ ]:
import librosa
import soundfile as sf

# Path to the dataset
dataset_path = '/kaggle/input/leaderspeeches/16000_pcm_speeches'

# Output directory to save the combined files
output_dir = '/kaggle/working/combined_files'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# List of speaker folders
speaker_folders = [
    "Benjamin_Netanyau",
    "Jens_Stoltenberg",
    "Julia_Gillard",
    "Magaret_Tarcher",
    "Nelson_Mandela"
]

# Number of files to combine for each speaker
num_files_to_combine = 200

# Iterate over each speaker's folder
for speaker_folder in speaker_folders:
    speaker_folder_path = os.path.join(dataset_path, speaker_folder)

    # List the first num_files_to_combine WAV files in the speaker's folder
    wav_files = [f"{i}.wav" for i in range(num_files_to_combine)]

    # Combine all WAV files into a single long file
    combined_audio = []
    for wav_file in wav_files:
        wav_file_path = os.path.join(speaker_folder_path, wav_file)
        audio, sr = librosa.load(wav_file_path, sr=None)
        combined_audio.extend(audio)

    # Save the combined audio file
    output_file_path = os.path.join(output_dir, f"{speaker_folder}_combined.wav")
    sf.write(output_file_path, combined_audio, sr)

print("Combination complete. Combined files saved in:", output_dir)

In [ ]:
from IPython.display import display, Audio

# Function to play audio file
def play_audio(audio_path):
    display(Audio(filename=audio_path))

# Play a specific combined audio file
speaker_folder = "Benjamin_Netanyau_combined"
audio_path = os.path.join(output_dir, f"{speaker_folder}.wav")
print(f"Click the play button to listen: {audio_path}")
play_audio(audio_path)

Visualization

In [ ]:
import librosa.display

# Function to plot the waveform, spectrogram, and MFCCs
def plot_audio_features(audio_path):
    # Load audio file
    y, sr = librosa.load(audio_path, sr=None)

    # Extract speaker name from the file path
    speaker_name = os.path.basename(audio_path).split('_')[0]

    # Plot the waveform
    plt.figure(figsize=(15, 10))
    plt.subplot(3, 1, 1)
    librosa.display.waveshow(y, sr=sr)
    plt.title(f'Waveform - {speaker_name}')

    # Plot the spectrogram
    plt.subplot(3, 1, 2)
    D = librosa.amplitude_to_db(librosa.stft(y), ref=np.max)
    librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='log')
    plt.colorbar(format='%+2.0f dB')
    plt.title(f'Spectrogram - {speaker_name}')

    # Plot the MFCCs
    plt.subplot(3, 1, 3)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    librosa.display.specshow(mfccs, x_axis='time')
    plt.colorbar()
    plt.title(f'MFCCs - {speaker_name}')

    plt.tight_layout()
    plt.show()

# Paths to the combined audio files
audio_paths = [
    '/kaggle/working/combined_files/Benjamin_Netanyau_combined.wav',
    '/kaggle/working/combined_files/Jens_Stoltenberg_combined.wav'
]

# Plot features for each audio file
for audio_path in audio_paths:
    plot_audio_features(audio_path)

Feature Extraction

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# Set the parent directory for speaker folders
parent_dir = "/kaggle/input/leaderspeeches/16000_pcm_speeches"

# List of speaker folders
speaker_folders = [
    "Benjamin_Netanyau",
    "Jens_Stoltenberg",
    "Julia_Gillard",
    "Magaret_Tarcher",
    "Nelson_Mandela"
]

def extract_features(parent_dir, speaker_folders):
    features = []
    labels = []

    for i, speaker_folder in enumerate(speaker_folders):
        speaker_folder_path = os.path.join(parent_dir, speaker_folder)

        for filename in os.listdir(speaker_folder_path):
            if filename.endswith(".wav"):
                file_path = os.path.join(speaker_folder_path, filename)
                audio, sr = librosa.load(file_path, sr=None, duration=1)
                mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
                
                # Normalize MFCC features
                mfccs = StandardScaler().fit_transform(mfccs)
                
                features.append(mfccs.T)
                labels.append(i)

    return np.array(features), np.array(labels)

# Extract features and labels
X, y = extract_features(parent_dir, speaker_folders)

MFCC

In [ ]:
for feature in X[:1]:
    print(feature)

Dataset- 80% for training, 10% for validation, 10% for testing

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder

# Encode labels with explicit classes
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
label_encoder.classes_ = np.array(speaker_folders)

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Print the shapes of training and validation data
print("Training Data Shape:", X_train.shape)
print("Validation Data Shape:", X_val.shape)
print("Testing Data Shape:", X_test.shape)

RNN

In [ ]:
# Define the RNN model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(speaker_folders), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Train the model with EarlyStopping
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=32, callbacks=[early_stopping])

# Check if EarlyStopping triggered
if early_stopping.stopped_epoch > 0:
    print("Early stopping triggered at epoch", early_stopping.stopped_epoch + 1)
else:
    print("Training completed without early stopping")

# Plot training vs validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Evaluation

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score

# Evaluate the model on the test set
y_pred_probabilities = model.predict(X_test)
y_pred = np.argmax(y_pred_probabilities, axis=1)

# Decode labels back to original format
y_test_decoded = label_encoder.inverse_transform(y_test)
y_pred_decoded = label_encoder.inverse_transform(y_pred)

# Create a confusion matrix
conf_matrix = confusion_matrix(y_test_decoded, y_pred_decoded, labels=speaker_folders)

# Calculate accuracy
accuracy = accuracy_score(y_test_decoded, y_pred_decoded)
print(f"Test Evaluation Accuracy: {accuracy}")

# Calculate F1 score
f1 = f1_score(y_test_decoded, y_pred_decoded, labels=speaker_folders, average='weighted')
print(f"Weighted F1 Score: {f1}")

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=speaker_folders, yticklabels=speaker_folders)

# Rotate x-axis labels by 45 degrees
plt.xticks(rotation=45, ha="right")

plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

# **CNN and MLP Model**

In [ ]:
import numpy as np
import pandas as pd
import os
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

# List of leaders' names
leaders = ['Nelson_Mandela', 'Benjamin_Netanyau', 'Magaret_Tarcher', 'Jens_Stoltenberg', 'Julia_Gillard']

# Path to the dataset
data_dir = '/kaggle/input/speaker-recognition-dataset/16000_pcm_speeches'

# Initialize lists to store file paths and labels
file_paths = []
labels = []

# Loop over the directories and files
for dirname, _, filenames in os.walk(data_dir):
    for filename in filenames:
        # Get the full file path
        file_path = os.path.join(dirname, filename)
        
        # Check if the file belongs to one of the leaders' classes
        speaker = os.path.basename(dirname)
        if speaker in leaders:
            file_paths.append(file_path)
            labels.append(speaker)

# Convert labels to numerical values
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(file_paths, labels, test_size=0.2, random_state=42)

# MFCC feature extraction
def extract_features(file_path):
    audio, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    mfccs = mfccs.T
    return mfccs

# Extract features for train and test sets
X_train_features = np.array([extract_features(file_path) for file_path in X_train])
X_test_features = np.array([extract_features(file_path) for file_path in X_test])

# Reshape the features for the CNN model
X_train_features = X_train_features.reshape(X_train_features.shape[0], X_train_features.shape[1], X_train_features.shape[2], 1)
X_test_features = X_test_features.reshape(X_test_features.shape[0], X_test_features.shape[1], X_test_features.shape[2], 1)

# Define the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=X_train_features.shape[1:]))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(leaders), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_features, y_train, epochs=50, batch_size=32, validation_data=(X_test_features, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test_features, y_test)
print(f'Test accuracy: {test_acc:.4f}')

In [ ]:
import numpy as np
import pandas as pd
import os
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten

# List of leaders' names
leaders = ['Nelson_Mandela', 'Benjamin_Netanyau', 'Magaret_Tarcher', 'Jens_Stoltenberg', 'Julia_Gillard']

# Path to the dataset
data_dir = '/kaggle/input/speaker-recognition-dataset/16000_pcm_speeches'

# Initialize lists to store file paths and labels
file_paths = []
labels = []

# Loop over the directories and files
for dirname, _, filenames in os.walk(data_dir):
    for filename in filenames:
        # Get the full file path
        file_path = os.path.join(dirname, filename)
        
        # Check if the file belongs to one of the leaders' classes
        speaker = os.path.basename(dirname)
        if speaker in leaders:
            file_paths.append(file_path)
            labels.append(speaker)

# Convert labels to numerical values
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(file_paths, labels, test_size=0.2, random_state=42)

# MFCC feature extraction
def extract_features(file_path):
    audio, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    mfccs = mfccs.T
    return mfccs

# Extract features for train and test sets
X_train_features = np.array([extract_features(file_path) for file_path in X_train])
X_test_features = np.array([extract_features(file_path) for file_path in X_test])

# Define the MLP model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_features.shape[1], X_train_features.shape[2])))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(len(leaders), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_features, y_train, epochs=50, batch_size=32, validation_data=(X_test_features, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test_features, y_test)
print(f'Test accuracy: {test_acc:.4f}')

In [ ]:
import numpy as np
import pandas as pd
import os
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten

# List of leaders' names
leaders = ['Nelson_Mandela', 'Benjamin_Netanyau', 'Magaret_Tarcher', 'Jens_Stoltenberg', 'Julia_Gillard']

# Path to the dataset
data_dir = '/kaggle/input/speaker-recognition-dataset/16000_pcm_speeches'

# Initialize lists to store file paths and labels
file_paths = []
labels = []

# Loop over the directories and files
for dirname, _, filenames in os.walk(data_dir):
    for filename in filenames:
        # Get the full file path
        file_path = os.path.join(dirname, filename)
        
        # Check if the file belongs to one of the leaders' classes
        speaker = os.path.basename(dirname)
        if speaker in leaders:
            file_paths.append(file_path)
            labels.append(speaker)

# Convert labels to numerical values
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(file_paths, labels, test_size=0.2, random_state=42)

# MFCC feature extraction
def extract_features(file_path):
    audio, sr = librosa.load(file_path, sr=None)
    stft = np.abs(librosa.stft(audio))
    return stft

# Extract features for train and test sets
X_train_features = np.array([extract_features(file_path) for file_path in X_train])
X_test_features = np.array([extract_features(file_path) for file_path in X_test])

# Define the MLP model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_features.shape[1], X_train_features.shape[2])))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(len(leaders), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_features, y_train, epochs=50, batch_size=32, validation_data=(X_test_features, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test_features, y_test)
print(f'Test accuracy: {test_acc:.4f}')